# Finetuning GPT-2 for generating product reviews

This colab is for fine-tuning GPT-2 on an Amazon review dataset using the [Hugging Face Transformer](https://github.com/huggingface/transformers) library.

**IMPORTANT: Set GPU as the Hardware Accelerator in `Runtime > Change runtime type` before running the Colab.**

## Setup

### HuggingFace Transfomers library install + library imports

In [0]:
!git clone https://github.com/huggingface/transformers
!git checkout b90745c

import os
os.chdir('/content/transformers')

!pip install .
!pip install -r ./examples/requirements.txt

os.chdir('/content/transformers/examples')

!pip install dict_to_obj


fatal: destination path 'transformers' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git
Processing /content/transformers
  Created wheel for transformers: filename=transformers-2.5.1-cp36-none-any.whl size=498745 sha256=c5fba42282270fe275f023f2f2bb48c51b55a4b9767322a7d49e4406322dc74c
  Stored in directory: /tmp/pip-ephem-wheel-cache-9g7te0ly/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 2.5.1
    Uninstalling transformers-2.5.1:
      Successfully uninstalled transformers-2.5.1


In [0]:
import torch
import run_language_modeling
import run_generation
from dict_to_obj import DictToObj
import collections
import random
import numpy as np

print(torch.cuda.is_available())

True


### Mount Google Drive
Use **UPenn** Google Drive because it has unlimited storage space.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (Optional) Obtain training, testing, and validation datasets for Amazon reviews.
Training dataset should be roughly 20x larger than testing and dataset size should be between 5 and 100 MB.

**This code will not run on colab unless you wget and unzip the Amazon review datasets from https://nijianmo.github.io/amazon/index.html and configure the directories properly.**

Running this code will automatically generate the three datasets.

In [0]:
# import json

# train_fout = open('reviews_train.txt', 'w')
# train_link_fout = open('reviews_train_link.txt', 'w')
# test_fout = open('reviews_test.txt', 'w')
# test_link_fout = open('reviews_test_link.txt', 'w')
# valid_fout = open('reviews_valid.txt', 'w')
# valid_link_fout = open('reviews_valid_link.txt', 'w')
# files_to_read = ["All_Beauty_5.json", 
#                  "Books_5.json", 
#                  "Clothing_Shoes_and_Jewelry_5.json", 
#                  "Digital_Music_5.json", 
#                  "Electronics_5.json", 
#                  "Movies_and_TV_5.json", 
#                  "Toys_and_Games_5.json", 
#                  "Video_Games_5.json"]

# for index in range(len(files_to_read)):
#     train_skips_per_review = 401
#     test_skips_per_review = 8209
#     valid_skips_per_review = 8017
#     if index == 0:
#         train_skips_per_review = 109
#         test_skips_per_review = 2063
#         valid_skips_per_review = 1901
#     elif index == 1:
#         train_skips_per_review = 2069
#         test_skips_per_review = 42797
#         valid_skips_per_review = 39733
#     elif index == 3:
#         train_skips_per_review = 223
#         test_skips_per_review = 4099
#         valid_skips_per_review = 4019
#     elif index == 7:
#         train_skips_per_review = 389
#         test_skips_per_review = 8243
#         valid_skips_per_review = 8069
#     fin = open(files_to_read[index], 'r')
#     raw_line = fin.readline()
#     count = 0
#     while len(raw_line) > 0:
#         count += 1
#         review_json = json.loads(raw_line)
#         if "reviewText" in review_json:
#             review_text = review_json["reviewText"] # extract review text
#             cleaned_review = review_text.replace('\n', ' ') # remove newlines
            
#             if len(cleaned_review) > 800 and count % train_skips_per_review == 0 and count % test_skips_per_review != 0 and count % valid_skips_per_review != 0:
#                 train_fout.write(cleaned_review + '\n')
#                 train_link_fout.write(cleaned_review + '\n' + "https://www.amazon.com/dp/" + review_json["asin"] + '\n')
            
#             if len(cleaned_review) > 800 and count % train_skips_per_review != 0 and count % test_skips_per_review == 0 and count % valid_skips_per_review != 0:
#                 test_fout.write(cleaned_review + '\n')
#                 test_link_fout.write(cleaned_review + '\n' + "https://www.amazon.com/dp/" + review_json["asin"] + '\n')
            
#             if len(cleaned_review) > 800 and count % train_skips_per_review != 0 and count % test_skips_per_review != 0 and count % valid_skips_per_review == 0:
#                 valid_fout.write(cleaned_review + '\n')
#                 valid_link_fout.write(cleaned_review + '\n' + "https://www.amazon.com/dp/" + review_json["asin"] + '\n')
#         raw_line = fin.readline()

### Download Amazon review data.
The resulting datasets from the above script can be downloaded to Colab.

In [0]:
# Download the train, test, and validation datasets.
!wget -nc -O /content/review_test.txt https://dl.dropbox.com/s/rp74kt5uhsc57jz/reviews_test.txt?dl=1
!wget -nc -O /content/review_valid.txt https://dl.dropbox.com/s/i8ym3hfwkd3zxuw/reviews_valid.txt?dl=1
!wget -nc -O /content/review_train.txt https://dl.dropbox.com/s/zqqfpx8pn6e9hd6/reviews_train.txt?dl=1
!wget -nc -O /content/review_test_link.txt https://dl.dropbox.com/s/khqxx4kw7hyrk2g/reviews_test_link.txt?dl=1
!wget -nc -O /content/review_valid_link.txt https://dl.dropbox.com/s/csmlwkkisozt3or/reviews_valid_link.txt?dl=1

File ‘/content/review_test.txt’ already there; not retrieving.
File ‘/content/review_valid.txt’ already there; not retrieving.
File ‘/content/review_train.txt’ already there; not retrieving.
File ‘/content/review_test_link.txt’ already there; not retrieving.
File ‘/content/review_valid_link.txt’ already there; not retrieving.


## Finetune and Eval
[run_language_modeling.py](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) will train and evaluate the language model.

Call the script directly from the command line in order to launch training; also we will use functions from this script to conduct evaluation and generate samples at inference time.

### Launch fine-tuninng
We will be calling `run_language_modeling.py` from the command line to launch fine-tuning, **Running fine-tuning may take several hours.** Every `save_steps` steps, a checkpoint is saved to disk. The checkpoint contains all the learned weights for your model, and you can  always reload the model from a saved checkpoint, even if your Colab has crashed.

Below is an explanation of some of the arguments you might want to modify in the command below. 

* `--line_by_line`: Add `--line_by_line` if distinct lines of the text should be treated as distinct training examples. For example, if your dataset contains one story/tweet/article per line, this should be set.
* `--num_train_epochs`: The number of times to iterate over the train set. Increasing the number of epochs may result in better performance, but making this number too high will cause the model to overfit on the train set.
* `--block_size`: Your training text is truncated into blocks of this length. At test time, you will only want to generate sequences that are at most this length.
* `--gradient_accumulation_steps`: Update the model weights every this many steps. You shold set this to >1 when the batch size is very small to improve training stability.
* `--output_dir`: This is the where checkpoints will get saved. When you finetune on your own dataset, you should change this path. We recommend saving checkpoints to your Google Drive (`/content/drive/My Drive/`) so you can access them even if the Colab session dies.
* `--model_name_or_path` The path to the model weights to use when starting fine-tuning. You can set this to `gpt2-medium` to initialize with GPT-2's 355 million parameter model, or `gpt2` to initialize with their smaller 124 million parameter model. You can also set this to one of your own checkpoints to restart your training job if it crashes.

**I am getting out-of memory errors. What do I do?**

The number of trainable paramters in the model is a function of the `block_size` and the `batch_size`. If you are getting out-of-memory errors, then try drecreasing these value.

**Oh no! My computer went to sleep and the Colab disconnected.**

The train job might have still completed. Check the `output_dir` in your Google Drive to see if checkpoint files have been created there.

**Training is taking foreverrrrrr.**

Try decreasing `num_train_epochs` or changing `model_name_or_path` to `gpt2` instead of `gpt2-medium`.
If your evaluation set is very large, you might also want to remove the `evaluate_during_training` flag or increase `logging_steps`.

In [0]:
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/finetuned_models/amazon_reviews' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=4.0 \
    --do_train \
    --evaluate_during_training \
    --logging_steps=500 \
    --line_by_line \
    --save_steps=500 \
    --train_data_file=/content/review_train.txt \
    --do_eval \
    --eval_data_file=/content/review_valid.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=128 \
    --gradient_accumulation_steps=5 \
    --overwrite_output_dir

### Compute perplexity of a dataset.
This section shows how to compute perplexity of a dataset according to either the pre-trained or your fine-tuned language model. While this is possible to do by calling `run_language_modeling.py` on the command-line as above, we'll instead call the Python functions directly.

#### Look at what checkpoints are available
Run `ls` to look at what checkpoints saved been saved. You'll want to set `CHECKPOINT_PATH` below to one of these in order to evaluate the model weights saved in that checkpoint.

In [0]:
!ls '/content/drive/My Drive/finetuned_models/amazon_reviews'

checkpoint-1000  checkpoint-500    pytorch_model.bin	    vocab.json
checkpoint-1500  config.json	   special_tokens_map.json
checkpoint-2000  eval_results.txt  tokenizer_config.json
checkpoint-2500  merges.txt	   training_args.bin


#### Helper functions

In [0]:
def load_model(args):
  """Creates a model and loads in weights for it."""
  config_class, model_class, _ = run_language_modeling.MODEL_CLASSES[args.model_type] 
  config = config_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  model = model_class.from_pretrained(
      args.model_name_or_path,
      from_tf=bool(".ckpt" in args.model_name_or_path),
      config=config,
      cache_dir=None,
  )
  model.to(args.device)
  return model

def set_seed(seed):
  """Set the random seed."""
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

def do_perplexity_eval(args, model, data_file_path):
  """Computes the perplexity of the text in data_file_path according to the provided model."""
  set_seed(args.seed)

  args.eval_data_file=data_file_path

  _, _, tokenizer_class = run_language_modeling.MODEL_CLASSES[args.model_type]
  tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  args.block_size = min(args.block_size, tokenizer.max_len)

  result = run_language_modeling.evaluate(args, model, tokenizer, prefix="")
  return result

#### Compute it.

In [0]:
# Set this to the checkpoint you want to evalute, or to "gpt2-medium" to
# evaluate the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/amazon_reviews/checkpoint-2500'
# CHECKPOINT_PATH = "gpt2-medium"

# Set this to the list of text files you want to evaluate the perplexity of.
DATA_PATHS = ["/content/review_valid.txt",
              "/content/review_test.txt"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  block_size = 128,
  local_rank=-1,
  eval_batch_size=2,
  per_gpu_eval_batch_size=2,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  line_by_line=False,
  overwrite_cache=None,
  model_type='gpt2',
  seed=42,
)
args = DictToObj(args)

model = load_model(args)

for data_path in DATA_PATHS:
  eval_results = do_perplexity_eval(args, model, data_path)
  perplexity = eval_results['perplexity']
  print('{} is the perplexity of {} according to {}'.format(
      perplexity, data_path, CHECKPOINT_PATH))

### Generate samples
The following code generates 5 text samples of length 800 characters for each review in the test dataset that are continuations of its first 140 characters.

In [0]:
def generate_samples(args, model, prompt_text):
  """Generating sampling for the provided prompt using the provided model."""
  set_seed(args.seed)

  _, _, tokenizer_class = run_language_modeling.MODEL_CLASSES[args.model_type]
  tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  requires_preprocessing = args.model_type in run_generation.PREPROCESSING_FUNCTIONS.keys()
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(args.device)

  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=args.length + len(encoded_prompt[0]),
      temperature=args.temperature,
      top_k=args.k,
      top_p=args.p,
      repetition_penalty=args.repetition_penalty,
      do_sample=True,
      num_return_sequences=args.num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(args.stop_token) if args.stop_token else None]

    # Remove the excess text that was used for pre-processing
    text = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]

    # Add the prompt at the beginning of the sequence.
    total_sequence = prompt_text + text

    generated_sequences.append(total_sequence)

  return generated_sequences

In [0]:
# Set this to the checkpoint you want to use for generation, or to "gpt2-medium"
# to generate with the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/amazon_reviews/checkpoint-500' #'gpt2-medium'

files_to_read = ['/content/review_train.txt']
# fout = open('/content/drive/My Drive/checkpoint-500-output.txt', 'w')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  model_type='gpt2',
  seed=42,
  stop_token=None, # Set this if your dataset has a special word that indicates the end of a text.
  temperature=1.0,  # temperature sampling. Set this to temperature=1.0 to not use temperature.
  k=50,  # k for top-k sampling. Set this to k=0 to not use top-k.
  p=1.0,  # p for nucleus sampling. Set this to p=1.0 to not use nucleus sampling.
  repetition_penalty=None,
  length=300,  # Number of tokens to generate.
  num_return_sequences=1,  # Number of independently computed samples to generate.
)
args = DictToObj(args)

model = load_model(args)

for file_to_read in files_to_read:
  fin = open(file_to_read, 'r')
  review_text = fin.readline()
  for i in range(0,85):
    review_text = fin.readline()
  count = 0
  while len(review_text) > 0 and count < 500:
    # review_link = fin.readline()
    PROMPT = review_text[:70]
    last_space = PROMPT.rfind(' ')
    if last_space != -1:
      PROMPT = PROMPT[:last_space]
    sequences = generate_samples(args, model, PROMPT)
    # print(review_link + '\n')
    for idx, sequence in enumerate(sequences):
      # print('====== GENERATION {} ======'.format(idx))
      full_review = sequence.replace('\n', '')
      if full_review.find('.') != -1 or full_review.find('!') != -1 or full_review.find('?') != -1:
        last_period_index = min(800, len(full_review))
        if last_period_index == 800:
          while full_review[last_period_index] != '.' and full_review[last_period_index] != '?' and full_review[last_period_index] != '!':
            last_period_index -= 1
        print(full_review[:last_period_index + 1] + '\n')
    review_text = fin.readline()
    count += 1